In [55]:
!pip install pandas polars numpy pyarrow

/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

If you really know what your doing, you can silence this warning with the warning module
or by setting POLARS_ALLOW_FORKING_THREAD=1.

  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 56.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [56]:
import duckdb
import numpy
import pandas as pd
import polars
import pyarrow

In [57]:
duckdb.sql("SELECT 42").show()

┌───────┐
│  42   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [58]:
duckdb.sql("SELECT 42").fetchall()   # Python objects
duckdb.sql("SELECT 42").df()         # Pandas DataFrame
duckdb.sql("SELECT 42").pl()         # Polars DataFrame
duckdb.sql("SELECT 42").arrow()      # Arrow Table
duckdb.sql("SELECT 42").fetchnumpy() # NumPy Arrays

{'42': array([42], dtype=int32)}

In [59]:
duckdb.sql("SELECT 42").fetchnumpy() # NumPy Arrays

{'42': array([42], dtype=int32)}

In [60]:
con = duckdb.connect()
con.install_extension("s3")
con.load_extension("s3")

con.install_extension("httpfs")
con.load_extension("httpfs")

In [61]:
CRAWL_SET = "CC-MAIN-2024-42"
CC_URL_S3_BASE="s3://"
CC_URL_HTTP_BASE="https://data.commoncrawl.org/"
CC_INDEX_URL = f"{CC_URL_HTTP_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_URL

'https://data.commoncrawl.org/commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [63]:
print(f"{CC_INDEX_S3_URL}")

s3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet


In [9]:
#!cut -d' ' -f3- ../cc/cdx-00000 > ../cc/cdx-00000.jsonl 

In [10]:
!duckdb -c ""

In [22]:
DB_FILE = "./cdx.duckdb"

In [23]:
!duckdb -c "\
  create table meta as SELECT * FROM '../cc/cdx-00000.jsonl'; \
  create view status_vw as Select status, count(status) as cnt from meta group by status order by cnt desc; \
  select * from meta; \
  select * from status_vw; \
" {DB_FILE}

100% ▕████████████████████████████████████████████████████████████▏ 
┌──────────────────────┬────────────┬───┬───────────┬───────────┐
│         url          │    mime    │ … │ languages │ truncated │
│       varchar        │  varchar   │   │  varchar  │  varchar  │
├──────────────────────┼────────────┼───┼───────────┼───────────┤
│ http://1.0.0.0/rob…  │ text/html  │ … │           │           │
│ http://1.0/robots.…  │ text/html  │ … │           │           │
│ http://165.22.100.0/ │ text/html  │ … │           │           │
│ http://165.22.100.…  │ text/html  │ … │           │           │
│ http://47.75.105.0…  │ text/html  │ … │ zho       │           │
│ http://47.75.105.0…  │ text/html  │ … │           │           │
│ http://34.236.106.…  │ text/html  │ … │ eng       │           │
│ http://34.236.106.…  │ text/html  │ … │           │           │
│ http://139.59.109.…  │ text/html  │ … │ eng       │           │
│ http://139.59.109.…  │ text/plain │ … │           │           │
│ http:

In [26]:
!duckdb -c ".databases" {DB_FILE}
#!duckdb -c ".open hello.db"

cdx: ./cdx.duckdb


In [43]:
!duckdb -c " \
create table metastats as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta group by tstatus order by cstatus desc; \
select * from metastats; \
" {DB_FILE}

┌─────────┬─────────┬────────┐
│ tstatus │ cstatus │  pct   │
│ varchar │  int64  │ double │
├─────────┼─────────┼────────┤
│ 200     │ 9104480 │  85.49 │
│ 301     │  514285 │   4.83 │
│ 404     │  489284 │   4.59 │
│ 302     │  311106 │   2.92 │
│ 304     │   52519 │   0.49 │
│ 308     │   49289 │   0.46 │
│ 307     │   42106 │    0.4 │
│ 500     │   16486 │   0.15 │
│ 403     │   13914 │   0.13 │
│ 303     │   13372 │   0.13 │
│ 410     │    9494 │   0.09 │
│ 406     │    6755 │   0.06 │
│ 401     │    5352 │   0.05 │
│ 503     │    4687 │   0.04 │
│ 502     │    3066 │   0.03 │
│ 429     │    2508 │   0.02 │
│ 522     │    2329 │   0.02 │
│ 400     │    2190 │   0.02 │
│ 520     │    2037 │   0.02 │
│ 402     │     880 │   0.01 │
│  ·      │       · │     ·  │
│  ·      │       · │     ·  │
│  ·      │       · │     ·  │
│ 499     │      10 │    0.0 │
│ 533     │       9 │    0.0 │
│ 415     │       9 │    0.0 │
│ 206     │       7 │    0.0 │
│ 444     │       7 │    0.0 │
│ 507   

In [67]:
!duckdb -c " \
create table metastats_norobotstxt as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta where url not like '%robots.txt' group by tstatus order by cstatus desc; \
select * from metastats_norobotstxt; \
" {DB_FILE}

┌─────────┬─────────┬────────┐
│ tstatus │ cstatus │  pct   │
│ varchar │  int64  │ double │
├─────────┼─────────┼────────┤
│ 200     │ 8906463 │  86.65 │
│ 301     │  478984 │   4.66 │
│ 404     │  388104 │   3.78 │
│ 302     │  299691 │   2.92 │
│ 304     │   52516 │   0.51 │
│ 308     │   46008 │   0.45 │
│ 307     │   39977 │   0.39 │
│ 500     │   13362 │   0.13 │
│ 303     │   11806 │   0.11 │
│ 403     │    9748 │   0.09 │
│ 410     │    9132 │   0.09 │
│ 401     │    5012 │   0.05 │
│ 406     │    4599 │   0.04 │
│ 503     │    3216 │   0.03 │
│ 502     │    2224 │   0.02 │
│ 400     │    1788 │   0.02 │
│ 520     │    1670 │   0.02 │
│ 429     │     857 │   0.01 │
│ 402     │     802 │   0.01 │
│ 522     │     702 │   0.01 │
│  ·      │       · │     ·  │
│  ·      │       · │     ·  │
│  ·      │       · │     ·  │
│ 422     │      21 │    0.0 │
│ 421     │      20 │    0.0 │
│ 412     │      17 │    0.0 │
│ 418     │      16 │    0.0 │
│ 416     │      15 │    0.0 │
│ 501   

In [ ]:
#!duckdb -c "drop table metastats" {DB_FILE}

In [22]:
first_line = ""
with open("../cc/cdx-00000", "r") as file:
    first_line = file.readline()

print(first_line)

0,0,0,1)/robots.txt 20241012004456 {"url": "http://1.0.0.0/robots.txt", "mime": "text/html", "mime-detected": "text/html", "status": "403", "digest": "UNFBLUUDKY6EPX25BG4X7TI6J3IS2BBZ", "length": "2706", "offset": "890", "filename": "crawl-data/CC-MAIN-2024-42/segments/1727944253858.36/robotstxt/CC-MAIN-20241011230131-20241012020131-00456.warc.gz"}



In [63]:
!echo "{first_line}" | cut -d' ' -f3- 

{url: http://1.0.0.0/robots.txt, mime: text/html, mime-detected: text/html, status: 403, digest: UNFBLUUDKY6EPX25BG4X7TI6J3IS2BBZ, length: 2706, offset: 890, filename: crawl-data/CC-MAIN-2024-42/segments/1727944253858.36/robotstxt/CC-MAIN-20241011230131-20241012020131-00456.warc.gz}



In [61]:
import json
s = '{"url": "http://1.0.0.0/robots.txt", "mime": "text/html", "mime-detected": "text/html", "status": "403", "digest": "UNFBLUUDKY6EPX25BG4X7TI6J3IS2BBZ", "length": "2706", "offset": "890", "filename": "crawl-data/CC-MAIN-2024-42/segments/1727944253858.36/robotstxt/CC-MAIN-20241011230131-20241012020131-00456.warc.gz"}'
json.loads(s)

{'url': 'http://1.0.0.0/robots.txt',
 'mime': 'text/html',
 'mime-detected': 'text/html',
 'status': '403',
 'digest': 'UNFBLUUDKY6EPX25BG4X7TI6J3IS2BBZ',
 'length': '2706',
 'offset': '890',
 'filename': 'crawl-data/CC-MAIN-2024-42/segments/1727944253858.36/robotstxt/CC-MAIN-20241011230131-20241012020131-00456.warc.gz'}

In [45]:
con.sql(f"select * from read_parquet('{CC_INDEX_URL}')")

HTTPException: HTTP Error: Unable to connect to URL "https://data.commoncrawl.org/commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet": 404 (Not Found)

In [ ]:
duckdb.read_parquet(CC_INDEX_URL)

In [64]:
!duckdb -c ".schema" {DB_FILE}

CREATE TABLE meta(url VARCHAR, mime VARCHAR, "mime-detected" VARCHAR, status VARCHAR, digest VARCHAR, length VARCHAR, "offset" VARCHAR, filename VARCHAR, redirect VARCHAR, charset VARCHAR, languages VARCHAR, truncated VARCHAR);
CREATE TABLE metastats(tstatus VARCHAR, cstatus BIGINT, pct DOUBLE);
CREATE VIEW status_vw AS SELECT status, count(status) AS cnt FROM meta GROUP BY status ORDER BY cnt DESC;


In [66]:
!duckdb -c "select * from meta where url like '%robots.txt' limit 3" {DB_FILE}

┌──────────────────────┬───────────┬───────────────┬───┬───────────┬───────────┐
│         url          │   mime    │ mime-detected │ … │ languages │ truncated │
│       varchar        │  varchar  │    varchar    │   │  varchar  │  varchar  │
├──────────────────────┼───────────┼───────────────┼───┼───────────┼───────────┤
│ http://1.0.0.0/rob…  │ text/html │ text/html     │ … │           │           │
│ http://1.0/robots.…  │ text/html │ text/html     │ … │           │           │
│ http://165.22.100.…  │ text/html │ text/html     │ … │           │           │
├──────────────────────┴───────────┴───────────────┴───┴───────────┴───────────┤
│ 3 rows                                                  12 columns (5 shown) │
└──────────────────────────────────────────────────────────────────────────────┘


In [69]:
!head -n 10 '../cc/cdx-00000.jsonl'

{"url": "http://1.0.0.0/robots.txt", "mime": "text/html", "mime-detected": "text/html", "status": "403", "digest": "UNFBLUUDKY6EPX25BG4X7TI6J3IS2BBZ", "length": "2706", "offset": "890", "filename": "crawl-data/CC-MAIN-2024-42/segments/1727944253858.36/robotstxt/CC-MAIN-20241011230131-20241012020131-00456.warc.gz"}
{"url": "http://1.0/robots.txt", "mime": "text/html", "mime-detected": "text/html", "status": "403", "digest": "GU4Y6L3AUIXVNMMNTWLI7DEQOYT6QOTG", "length": "2703", "offset": "2506", "filename": "crawl-data/CC-MAIN-2024-42/segments/1727944253306.52/robotstxt/CC-MAIN-20241004103433-20241004133433-00732.warc.gz"}
{"url": "http://165.22.100.0/", "mime": "text/html", "mime-detected": "text/html", "status": "301", "digest": "DCNYNIFG5SBRCVS5PCUY4YY2UM2WAQ4R", "length": "690", "offset": "12101", "filename": "crawl-data/CC-MAIN-2024-42/segments/1727944253371.47/crawldiagnostics/CC-MAIN-20241005143707-20241005173707-00443.warc.gz", "redirect": "https://157.245.55.71/"}
{"url": "http:

In [76]:
import gzip

In [82]:
with open('./examples/CC-MAIN-20241014005649-20241014035649-00707.warc', 'rb') as f:
  file_content = f.read(10000).decode('utf-8')
  print(file_content)

WARC/1.0
WARC-Type: warcinfo
WARC-Date: 2024-10-14T00:56:49Z
WARC-Record-ID: <urn:uuid:2d4f52d6-aad5-42da-9be0-8f621c9ae90b>
Content-Length: 490
Content-Type: application/warc-fields
WARC-Filename: CC-MAIN-20241014005649-20241014035649-00707.warc.gz

isPartOf: CC-MAIN-2024-42
publisher: Common Crawl
description: Wide crawl of the web for October 2024
operator: Common Crawl Admin (info@commoncrawl.org)
hostname: ip-10-67-67-116
software: Apache Nutch 1.20 (modified, https://github.com/commoncrawl/nutch/)
robots: checked via crawler-commons 1.5-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)
format: WARC File Format 1.1
conformsTo: https://iipc.github.io/warc-specifications/specifications/warc-format/warc-1.1/


WARC/1.0
WARC-Type: request
WARC-Date: 2024-10-14T01:37:39Z
WARC-Record-ID: <urn:uuid:ad52513e-a3ea-40af-9c1d-c30865ba976e>
Content-Length: 269
Content-Type: application/http; msgtype=request
WARC-Warcinfo-ID: <urn:uuid:2d4f52d6-aad5-42da-9be0-8f621c9ae90b>
WARC-IP-

In [83]:
import warcio